## Lobes

- Measure fluxes from different regions at multiple frequencies
- The r.m.s. in the input images should be measured in a different notebook and saved to the input table, since for the Planck images we use a different method.
- MeerKAT observations must be corrected for the primary beam BEFORE measuring the flux
- Planck observations must be converted to the correct units BEFORE measuring the flux
- **Delete** old flux tables before repeaing measurement


In [1]:
import numpy as np
import shutil
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
from prettytable import PrettyTable



fint = fluxInt.flInt()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

rootDir = '/Users/maccagni/Projects/MFS/FornaxA/contData/'
regDir = rootDir+'regions/'
imDir = rootDir+'inIms/'

names = ['ToT','ELobe','WLobe','Core']

ToTDir = rootDir+'ToT/'
ELobeDir = rootDir+'ELobe/'
WLobeDir = rootDir+'WLobe/'


fint.rootdir = rootDir
syn.rootdir = rootDir
pb.rootdir = rootDir


tableName = rootDir+'inputFluxTable.tbl'
tableFileNames= ascii.read(tableName,format='csv')


noiseTableName = rootDir+'noiseFluxInt.tbl'
noiseValues= ascii.read(noiseTableName)

print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''



    ERROR: Montage commands could not be found.

    In order to use the montage_wrapper module, you will first need to
    install the IPAC Montage software from:

        http://montage.ipac.caltech.edu

    and ensure that the Montage commands (e.g. mAdd, mProject, etc.) are in
    your $PATH. Your current $PATH variable contains the following paths,
    but none of them contain the Montage commands:

        PATH = /Users/maccagni/programs/anaconda2/bin:/usr/local/karma/cm_sc
           ript:/usr/local/karma/csh_script:/usr/local/karma/site/x86_64
           _Darwin/bin:/usr/local/karma/bin:/Users/maccagni/programs/rad
           iobs:/usr/local/Cellar/gcc/8.2.0/bin:/usr/local/opt/inetutils
           /libexec/gnubin:/Users/maccagni/programs/patchelf-0.9/bin:/Us
           ers/maccagni/programs/anaconda2/envs:/Users/maccagni/programs
           /anaconda2/bin:/Users/maccagni/programs/:/usr/local/bin:/usr/
           local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/
   

SystemExit: 1

/Users/maccagni/programs/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


We measure the flux in 24 frequency intervals  between 84 MHz and 217 GHz.

In [4]:
totReg = regDir+'ToT.reg'
elReg = regDir+'EL.reg'
wlReg = regDir+'WL.reg'

regions = [totReg, elReg, wlReg]

outTables = ['ToT/ToTFluxes.tbl','ELobe/ELFluxes.tbl','WLobe/WLFluxes.tbl']

flCorr = np.array(tableFileNames.columns[1],dtype=float)
errPerc=  np.array(tableFileNames.columns[2],dtype=float)


print '\t Executing FileList+Region Combo' 

fr=[]
flIntArray= np.zeros([len(regions),len(tableFileNames.columns[0])])
flErrArray= np.zeros([len(regions),len(tableFileNames.columns[0])])

cutoff= np.zeros([len(tableFileNames.columns[0])])

for i in xrange(0,len(tableFileNames.columns[0])):

    fileName = imDir+tableFileNames.columns[0][i]

    datas,heads = fint.openFile(fileName)
    datas=np.squeeze(datas)
    heads=fint.cleanHead(heads)

    if not 'CDELT2' in heads:
        heads['CDELT2'] = heads['CD2_2']
        heads['CDELT1'] = heads['CD1_1']
    if 'Frequency' in tableFileNames.dtype.names:
        freq = float(tableFileNames.columns[1][i])*1e6
        fr.append(freq)
    elif 'FREQ' in heads:
        freq = float(heads['FREQ'])
        fr.append(freq)
    elif 'CRVAL3' in heads:
        freq = float(heads['CRVAL3'])
        fr.append(freq)
    else:
        print fileName

    for j in xrange (0,len(regions)):
        print fileName
        
        if j == 0:
            maskedDataToT, background, noise, pixels=fint.maskDatReg(datas,heads,regions[j],cutoff[i])
            fluxint, numPixBeam =fint.measFlux(maskedDataToT,heads,errPerc[i])
            maskedDataEL = maskedDataToT.copy()
            maskedDataWL = maskedDataToT.copy()
        elif j ==1:
            maskedData, background, noise, pixels = fint.maskDatReg(maskedDataEL,heads,regions[j],cutoff[j])
            fluxint, numPixBeam =fint.measFlux(maskedData,heads,errPerc[i])
        elif j ==2:
            maskedData, background, noise, pixels = fint.maskDatReg(maskedDataWL,heads,regions[j],cutoff[j])
            fluxint, numPixBeam =fint.measFlux(maskedData,heads,errPerc[i])
        
        fluxint *= flCorr[i]
    
        flIntArray[j,i] = fluxint
        t,flErr = fint.writeTable(heads,fluxint,noiseValues.columns[6+j][i],numPixBeam,freq,errPerc[i],outTables[j])
        flErrArray[j,i] = flErr

tName = syn.writeSynageTable(fr,flIntArray[0,:],flErrArray[0,:],'ToT')
shutil.move(tName,ToTDir+'synageFl'+'ToT.tbl')

tName = syn.writeSynageTable(fr,flIntArray[1,:],flErrArray[1,:],'ELobe')
shutil.move(tName,ELobeDir+'synageFl'+'ELobe.tbl')

tName = syn.writeSynageTable(fr,flIntArray[2,:],flErrArray[2,:],'WLobe')
shutil.move(tName,WLobeDir+'synageFl'+'WLobe.tbl')

print '''\t+---------+\n\t Fluxes measured\n\t Tables Saved\n\t+---------+'''
  

	 Executing FileList+Region Combo
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_84MHz.fits
0.648398556028
157.5766447352621
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_84MHz.fits
0.464261227442
55.307778301146364
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_84MHz.fits
0.433581584396
100.90059418156721
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_105MHz.fits
0.490927058834
138.39358540240758
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_105MHz.fits
0.351472279434
47.55286134826479
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_105MHz.fits
0.328289646393
89.60103849935929
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_155MHz.fits
0.23291810931
124.92421715553625
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_155MHz.fits
0.166695929167
46.045762463903856
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_155MHz.fits
0.155830000003
77.07878173254205
/home/maccagni/Projects/MFS/FornaxA/contData/inIms/MWA_200MHz.fit

**Output tables**:

```
~/Projects/MFS/FornaxA/contData/ToT/ToTFluxes.tbl'
~/Projects/MFS/FornaxA/contData/ELobe/ToTFluxes.tbl'
~/Projects/MFS/FornaxA/contData/WLobe/ToTFluxes.tbl'

~/Projects/MFS/FornaxA/contData/ToT/synageFlToT.tbl'
~/Projects/MFS/FornaxA/contData/ELobe/synageFlELobe.tbl'
~/Projects/MFS/FornaxA/contData/WLobe/synageFlWLobe.tbl'

```

**...Run synage to analyse the spectral energy distribution...**

```
~/programs/synage/SYNAGE
cd ToT
exec synageToT.script
cd ELobe
exec synageELobe.script
cd WLobe
exec synageWLobe.script

```

**Output SYNAGE tables (in synOut/)**:

```
ToTCI_point1_fit.dat
ToTCIOFF_point1_fit.dat
ToTCI.stats
ToTCIOFF.stats

ELobeCI_point1_fit.dat
ELobeCIOFF_point1_fit.dat
ELobeCI.stats
ELobeCIOFF.stats

WLobeCI_point1_fit.dat
WLobeCIOFF_point1_fit.dat
WLobeCI.stats
WLobeCIOFF.stats

```
